# Fussy.jl

A Fusion Systems Code

---------

#### GUI includes:

 1. Model Comparisons 
 2. Sensitivity Studies
 3. Monte Carlo Samplings

#### Designs covered:

 Reactor | <span></span> | <span></span> | Affiliation | <span></span> | <span></span> | Info
-|-|-|-|-|-|-
Scylla | <span></span> | <span></span> | –  | <span></span> | <span></span> | Steady-State Prototype
Charybdis | <span></span> | <span></span> | – | <span></span> | <span></span> | Pulsed Prototype
ARC | <span></span> | <span></span> | MIT | <span></span> | <span></span> | Compact Steady-State Tokamak
ACT I | <span></span> | <span></span> | ARIES (US) | <span></span> | <span></span> | Optimistic Design
ACT II | <span></span> | <span></span> | ARIES (US) | <span></span> | <span></span> | Conservative Design
DEMO Steady | <span></span> | <span></span> | PROCESS (EU) | <span></span> | <span></span> | Steady-State Machine
DEMO Pulsed | <span></span> | <span></span> | PROCESS (EU) | <span></span> | <span></span> | Pulsed Machine

<br>
<hr>
<span class="js-load-message cs-loading">// loading should only take a minute.</span>

In [ ]:
cur_script = """
    <style class="js-load-remove-tag">
        .cs-loading:after {
            overflow: hidden;
            display: inline-block;
            vertical-align: bottom;
            -webkit-animation: cs-ellipsis steps(3,end) 3250ms infinite;      
            animation: cs-ellipsis steps(3,end) 3250ms infinite;
            content: "\\2026";
            width: 0px;
        }
        @keyframes cs-ellipsis { to { width: 0.9em; } }
        @-webkit-keyframes cs-ellipsis { to { width: 0.9em; } }
    </style>

    <script class="js-load-remove-tag">
      \$(".js-load-remove-tag").parent(".output_subarea").css("padding", "0");
    </script>
"""

display(HTML(cur_script))

In [1]:
using Fussy

using FileIO
using JLD2

using Plots
using Interact

using DataStructures
using StringCases

using LaTeXStrings

use_latex = true

if use_latex
    pgfplots()
else
    pyplot()
end

return

In [2]:
function toggle_buttons(show_buttons, hide_buttons)    
    cur_script = """
        <script class="js-toggle">
    """
    
    for cur_index in hide_buttons
    
        if cur_index < 0 
            cur_string = """
                \$(".jupyter-widgets").closest(".output_area").length
            """
        else
            cur_string = "0"
        end

        cur_script *= """
                \$(".jupyter-widgets").closest(".output_area").eq($(cur_string)+($(cur_index))).hide()
                \$(".jupyter-widgets").closest(".output_area").eq($(cur_string)+($(cur_index))).next().hide()
        """
        
    end
    
    for cur_index in show_buttons
    
        if cur_index < 0 
            cur_string = """
                \$(".jupyter-widgets").closest(".output_area").length
            """
        else
            cur_string = "0"
        end

        cur_script *= """
                \$(".jupyter-widgets").closest(".output_area").eq($(cur_string)+($(cur_index))).show()
                \$(".jupyter-widgets").closest(".output_area").eq($(cur_string)+($(cur_index))).next().show()
        """
        
    end
    
    cur_script *= """
            \$(".js-toggle").parent(".output_subarea").parent(".output_area").remove();
        </script>
    """

    display(HTML(cur_script)) 
end

return

In [3]:
@manipulate for loading=[true,false]
end

toggle_buttons([],[0])
return

Interact.Options{:ToggleButtons,Bool}(1: "input" = true Bool , "loading", true, "true", 1, Interact.OptionDict(DataStructures.OrderedDict("true"=>true,"false"=>false), Dict(false=>"false",true=>"true")), Any[], Any[], true, "horizontal", true)

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(0+(0)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(0+(0)).next().hide()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

In [4]:
cur_scans

LoadError: [91mUndefVarError: cur_scans not defined[39m

In [5]:
cur_scans = load("data/comparisons.jld2", "cur_scans")
cur_scans_demo = load("data/comparisons_demo_tau.jld2", "cur_scans_demo")

cur_scans[:demo_steady_modified] = cur_scans_demo[:demo_steady]
cur_scans[:demo_pulsed_modified] = cur_scans_demo[:demo_pulsed]

cur_studies = load("data/sensitivities.jld2", "cur_studies")
cur_samplings = load("data/samplings.jld2", "cur_samplings") 

return

In [6]:
cur_decks = [
    :proteus, # pulsed
    :charybdis, # steady state
    :arc, :act_1, :act_2,
    :demo_steady, :demo_pulsed
]

cur_params = [ 
    :T_bar, :n_bar, :I_P, :R_0, :B_0, 
    :tau_E, :p_bar, :eta_CD, :P_F, 
    :beta_N, :q_95, :q_DV, :P_W, 
    :norm_beta_N, :norm_q_95, :norm_P_E, :norm_P_W, 
    :f_BS, :f_CD, :f_IN, :W_M, :cost, 
    :a, :b, :c, :d, :h_CS, :R_CS, :volume, :rho_m
]

expanded_params = deepcopy(cur_params)

append!(
    expanded_params, 
    [
        :H, :Q, :wave_theta,
        :epsilon, :delta_95,
        :nu_n, :nu_T, :l_i,
        :N_G, :f_D, :Z_eff,
        :eta_CD, :B_CS, :tau_FT
    ]
)

x_list = deepcopy(cur_params)
y_list = deepcopy(cur_params)

xx_list = deepcopy(expanded_params)
yy_list = deepcopy(expanded_params)

xxx_list = [ 
    :H, :Q, :wave_theta,
    :epsilon, :kappa_95, :delta_95,
    :nu_n, :nu_T, :l_i,
    :N_G, :f_D, :Z_eff,
    :max_beta_N, :max_q_95, :max_P_W, 
    :eta_CD, :B_CS, :tau_FT
]

yyy_list = deepcopy(y_list)

Fussy.sort_lists!(map(lowercase, map(string,x_list)), x_list)
Fussy.sort_lists!(map(lowercase, map(string,xx_list)), xx_list)

Fussy.sort_lists!(map(lowercase, map(string,y_list)), y_list)
Fussy.sort_lists!(map(lowercase, map(string,yy_list)), yy_list)
Fussy.sort_lists!(map(lowercase, map(string,yyy_list)), yyy_list)

filter!(tmp_x -> tmp_x != :B_0, x_list)
filter!(tmp_y -> tmp_y != :R_0, y_list)

unshift!(x_list, :B_0)
unshift!(y_list, :R_0)

filter!(tmp_x -> tmp_x != :B_0, xx_list)
filter!(tmp_y -> tmp_y != :cost, yy_list)

unshift!(xx_list, :B_0)
unshift!(yy_list, :cost)

filter!(tmp_y -> tmp_y != :h_CS, yyy_list)
unshift!(yyy_list, :h_CS)

return

In [7]:
function fix_lims!(cur_lims, x, y, xscale, yscale, deck)
    cur_lims[1:2:3] /= 1.2
    cur_lims[2:2:4] *= 1.2
    
    if x == :cost
        cur_lims[1:2] = [0.001, 0.1]
    end
    
    if y == :cost
        cur_lims[3:4] = [0.001, 0.1]
    end
  
    if x == :W_M
        cur_lims[1:2] = [0.01, 100]
    end
    
    if y == :W_M
        cur_lims[3:4] = [0.01, 100]
    end
    
    if x == :B_0 || x == :R_0 || x == :h_CS
        cur_lims[1:2] = [0.05, 20]
    end
    
    if y == :B_0 || y == :R_0 || y == :h_CS
        cur_lims[3:4] = [0.05, 20]
    end
    
    if x == :T_bar || x == :I_P
        cur_lims[1:2] = [0.1, 40]
    end
  
    if y == :T_bar || y == :I_P
        cur_lims[3:4] = [0.1, 40]
    end
    
    if x == :eta_CD
        cur_lims[1:2] = [0.01, 0.4]
    end
  
    if y == :eta_CD
        cur_lims[3:4] = [0.01, 0.4]
    end
    
    if x == :kappa_95
        cur_lims[1:2] = [0.01, 3]
    end
  
    if y == :kappa_95
        cur_lims[3:4] = [0.01, 3]
    end
    
    if x == :P_F
        cur_lims[1:2] = [1, in(deck, [:act_2, :demo_steady]) ? 9000 : 4000 ]
    end
  
    if y == :P_F
        cur_lims[3:4] = [1, in(deck, [:act_2, :demo_steady]) ? 9000 : 4000 ]
    end
    
    
    if in(x, [:f_IN, :f_BS, :f_CD, :norm_P_W, :norm_beta_N, :norm_q_95])
         cur_lims[2] = 1.25
    end
    
    if in(y, [:f_IN, :f_BS, :f_CD, :norm_P_W, :norm_beta_N, :norm_q_95])
         cur_lims[4] = 1.25
    end
    
    if xscale == "log"
        iszero(cur_lims[1]) || plot!( xscale = :log )
    else
        cur_lims[1] = 0
    end
    
    if yscale == "log"
        iszero(cur_lims[3]) || plot!( yscale = :log )
    else
        cur_lims[3] = 0
    end
end

return

In [8]:
function filter_reactors!(reactor_list)
    filter!(Fussy.is_present, reactor_list)
    
    filter!(tmp_reactor -> tmp_reactor.is_valid, reactor_list)
    filter!(tmp_reactor -> tmp_reactor.is_good, reactor_list)
    
    filter!(tmp_reactor -> tmp_reactor.R_0 < 50, reactor_list)
    filter!(tmp_reactor -> tmp_reactor.B_0 < 50, reactor_list)

    filter!(tmp_reactor -> tmp_reactor.cost < 1, reactor_list)
    
    isempty(reactor_list) && return
    ( reactor_list[1].deck == :demo_steady ) && return
    ( reactor_list[1].deck == :act_1 ) && return
    
    filter!(tmp_reactor -> tmp_reactor.P_F <= 1.2 * ( in(tmp_reactor.deck, [:act_2, :demo_steady]) ? 9000 : 4000 ), reactor_list)
    filter!(tmp_reactor -> tmp_reactor.norm_P_E < 0.8, reactor_list)
end

return

In [9]:
function make_comparisons(deck, x, y, xscale, yscale)  
    is_pulsed = deck == :proteus || startswith(string(deck), "demo")
    
    if deck == :proteus
        other_label = "simple"
    elseif startswith(string(deck), "demo")
        other_label = "modified"
    else
        other_label = "steady"
    end
    
    other_type = Symbol("$(deck)_$(other_label)")
    
    min_x, max_x = Inf, -Inf
    min_y, max_y = Inf, -Inf
    
    scan_types = [deck, other_type]
    
    is_pulsed || reverse!(scan_types)
    
    for scan_type in scan_types
    
        cur_scan = deepcopy(cur_scans[scan_type])

        cur_dict = OrderedDict()

        cur_dict[:kink] = cur_scan.kink_reactors
        cur_dict[:beta] = cur_scan.beta_reactors
        cur_dict[:wall] = cur_scan.wall_reactors

        for (cur_key, cur_value) in cur_dict
            filter_reactors!(cur_value)

            isempty(cur_value) && continue

            min_branch_id = minimum(map(tmp_reactor -> tmp_reactor.branch_id, cur_value))
            max_branch_id = maximum(map(tmp_reactor -> tmp_reactor.branch_id, cur_value))

            for cur_branch_id in min_branch_id:max_branch_id
                tmp_value = filter(tmp_reactor -> tmp_reactor.branch_id == cur_branch_id, cur_value)
                ( length(tmp_value) > 1 ) || continue

                tmp_tt = map(tmp_reactor -> tmp_reactor.T_bar, tmp_value)

                tmp_xx = map(tmp_reactor -> getfield(tmp_reactor, x), tmp_value)
                tmp_yy = map(tmp_reactor -> getfield(tmp_reactor, y), tmp_value)

                Fussy.sort_lists!(tmp_tt, tmp_xx, tmp_yy)
                
                cur_label = string(cur_key)
                
                if scan_type == other_type 
                    cur_label = "$(other_label) - $(cur_label)"
                    cur_style = is_pulsed ? :dash : :solid
                else
                    cur_label = "pulsed - $(cur_label)"
                    cur_style = is_pulsed ? :solid : :dash
                end
                
                min_x = min(min_x, minimum(tmp_xx))
                min_y = min(min_y, minimum(tmp_yy))
                
                max_x = max(max_x, maximum(tmp_xx))
                max_y = max(max_y, maximum(tmp_yy))
                
                cur_label = join(map(capitalize, split(cur_label, " - ")), " - ")
                plot!(tmp_xx, tmp_yy, label=cur_label, style=cur_style)
            end
        end
        
    end
    
    cur_lims = [min_x, max_x, min_y, max_y]
    
    fix_lims!(cur_lims, x, y, xscale, yscale, deck)
    
    if isdefined(Fussy, Symbol("$(deck)_solution"))
        cur_solution = getfield(Fussy, Symbol("$(deck)_solution"))()
        tmp_x = getfield(cur_solution, x)
        tmp_y = getfield(cur_solution, y)
        
        ( tmp_x == nothing ) && ( tmp_x = NaN )
        ( tmp_y == nothing ) && ( tmp_y = NaN )
        
        plot!(cur_lims[1:2], [tmp_y, tmp_y], color=:black, opacity=0.5, label="", style=:dot, width=2)
        plot!([tmp_x, tmp_x], cur_lims[3:4], color=:black, opacity=0.5, label="", style=:dot, width=2)
        scatter!([tmp_x], [tmp_y], color=:black, opacity=0.5, label="")
    end
    
    return cur_lims
end

return

In [10]:
function make_sensitivities(deck, x, y, xscale, yscale)
    cur_study = deepcopy(cur_studies[x][deck])
    
    min_x, max_x = Inf, -Inf
    min_y, max_y = Inf, -Inf
    
    cur_dict = OrderedDict()

    cur_dict[:kink] = cur_study.kink_reactors
    cur_dict[:wall] = cur_study.wall_reactors
    cur_dict[:cost] = cur_study.cost_reactors
    cur_dict[:W_M] = cur_study.W_M_reactors
    
    for (cur_index, (cur_key, cur_value)) in enumerate(cur_dict)
        filter_reactors!(cur_value)
        
        isempty(cur_value) && continue
        
        tmp_xx = map(tmp_reactor -> getfield(tmp_reactor, x), cur_value)
        tmp_yy = map(tmp_reactor -> getfield(tmp_reactor, y), cur_value)

        min_x = min(min_x, minimum(tmp_xx))
        min_y = min(min_y, minimum(tmp_yy))

        max_x = max(max_x, maximum(tmp_xx))
        max_y = max(max_y, maximum(tmp_yy))
        
        if cur_index < 3
            cur_width = 3
            cur_style = :dot
        else
            cur_width = 1
            cur_style = :solid
        end

        plot!(tmp_xx, tmp_yy, label=latexstring(cur_key), color=cur_index, width=cur_width, style=cur_style, opacity=0.7)
    end

    study_mid_x = cur_study.default
    study_min_x = cur_study.default * ( 1 - cur_study.sensitivity )
    study_max_x = cur_study.default * ( 1 + cur_study.sensitivity )

    min_x = min(min_x, study_min_x)
    max_x = min(max_x, study_max_x)
    
    cur_lims = [min_x, max_x, min_y, max_y]
    
    fix_lims!(cur_lims, x, y, xscale, yscale, deck)

    plot!([study_mid_x, study_mid_x], cur_lims[3:4], color=length(cur_dict)+1, style=:dash, label="")
    
    plot!([study_min_x, study_min_x], cur_lims[3:4], color=length(cur_dict)+2, style=:dash, label="")
    plot!([study_max_x, study_max_x], cur_lims[3:4], color=length(cur_dict)+2, style=:dash, label="")
    
    return cur_lims
end

return

In [11]:
function make_samplings(deck, x, y, xscale, yscale, simple)
    cur_sampling = deepcopy(cur_samplings[deck])

    min_x, max_x = Inf, -Inf
    min_y, max_y = Inf, -Inf

    for (cur_key, cur_value) in cur_samplings
        for cur_index in 1:cur_value.study_count
            if Fussy.is_present(cur_value.wall_reactors[cur_index]) && Fussy.is_present(cur_value.cost_reactors[cur_index])
                cur_error = abs(cur_value.wall_reactors[cur_index].cost-cur_value.cost_reactors[cur_index].cost)
                ( cur_error < 1e-4 ) && ( cur_value.cost_reactors[cur_index] = nothing )
            end
            if Fussy.is_present(cur_value.cost_reactors[cur_index]) && Fussy.is_present(cur_value.W_M_reactors[cur_index])
                cur_error = abs(cur_value.cost_reactors[cur_index].cost-cur_value.W_M_reactors[cur_index].cost)
                ( cur_error < 1e-4 ) && ( cur_value.cost_reactors[cur_index] = nothing )
            end
            if Fussy.is_present(cur_value.kink_reactors[cur_index]) && Fussy.is_present(cur_value.W_M_reactors[cur_index])
                cur_error = abs(cur_value.kink_reactors[cur_index].cost-cur_value.W_M_reactors[cur_index].cost)
                ( cur_error < 1e-4 ) && ( cur_value.W_M_reactors[cur_index] = nothing )
            end
            if Fussy.is_present(cur_value.kink_reactors[cur_index]) && Fussy.is_present(cur_value.cost_reactors[cur_index])
                cur_error = abs(cur_value.kink_reactors[cur_index].cost-cur_value.cost_reactors[cur_index].cost)
                ( cur_error < 1e-4 ) && ( cur_value.cost_reactors[cur_index] = nothing )
            end
            if Fussy.is_present(cur_value.wall_reactors[cur_index]) && Fussy.is_present(cur_value.W_M_reactors[cur_index])
                cur_error = abs(cur_value.wall_reactors[cur_index].cost-cur_value.W_M_reactors[cur_index].cost)
                ( cur_error < 1e-4 ) && ( cur_value.W_M_reactors[cur_index] = nothing )
            end
            if Fussy.is_present(cur_value.wall_reactors[cur_index]) && Fussy.is_present(cur_value.kink_reactors[cur_index])
                cur_error = abs(cur_value.wall_reactors[cur_index].cost-cur_value.kink_reactors[cur_index].cost)
                ( cur_error < 1e-4 ) && ( cur_value.wall_reactors[cur_index] = nothing )
            end
        end
    end

    markers = [:pentagon, :square, :circle, :diamond, :hexagon]
    
    if x == :cost || y == :cost
        if x == :W_M || y == :W_M
            z = :P_F
            w = :P_F
        elseif x == :P_F || y == :P_F
            z = :W_M
            w = :W_M
        else
            z = :W_M
            w = :P_F
        end
    else
        z = :cost
        w = :W_M
    end
    
    limits = true
    constraints = true
    
    cur_list = [:cost, :W_M,:kink, :wall]
    
    this_xx = []
    this_yy = []
    this_zz = []
    this_ww = []
    this_ss = []
    this_mm = []
    this_oo = []
    this_gg = []
    
    for (tmp_index, kind) in enumerate(cur_list)
        tmp_reacs = deepcopy(getfield(cur_samplings[deck], Symbol("$(kind)_reactors")))
        
        filter_reactors!(tmp_reacs)
        
        isempty(tmp_reacs) && continue
        
        cur_xx = map(tmp_reac -> getfield(tmp_reac, x), tmp_reacs)
        cur_yy = map(tmp_reac -> getfield(tmp_reac, y), tmp_reacs)
        cur_zz = map(tmp_reac -> getfield(tmp_reac, z), tmp_reacs)
        cur_ww = map(tmp_reac -> getfield(tmp_reac, w), tmp_reacs)
        
        cur_mm = []
        cur_gg = []
        
        for tmp_reac in tmp_reacs
            if simple 
                if isapprox(tmp_reac.norm_P_W,1.0,rtol=1e-4)
                    if isapprox(tmp_reac.norm_beta_N,1.0,rtol=1e-4)
                        tmp_m = markers[4]
                        tmp_g = 4
                    else
                        tmp_m = markers[5]
                        tmp_g = 5
                    end
                elseif isapprox(tmp_reac.norm_q_95,1.0,rtol=1e-4)
                    if isapprox(tmp_reac.norm_beta_N,1.0,rtol=1e-4)
                        tmp_m = markers[2]
                        tmp_g = 2
                    else
                        tmp_m = markers[1]
                        tmp_g = 1
                    end
                else
                    @assert isapprox(tmp_reac.norm_beta_N,1.0,rtol=1e-4)
                    tmp_m = markers[3]
                    tmp_g = 3
                end
            else
                tmp_g = tmp_index
                tmp_m = markers[tmp_index]
            end
            push!(cur_mm, tmp_m)
            push!(cur_gg, tmp_g)
        end
    
        append!(this_oo, (tmp_index < 3 ? Int(limits) : Int(constraints))*ones(length(cur_zz)))
        append!(this_mm, cur_mm)
        append!(this_gg, cur_gg)
        append!(this_ss, map(tmp_m -> tmp_m ==:square ? 4/sqrt(2) : 4, cur_mm))

        append!(this_xx, cur_xx)
        append!(this_zz, cur_zz)
        append!(this_yy, cur_yy)
        append!(this_ww, cur_ww)
    end
    
    max_x = maximum(this_xx)
    max_y = maximum(this_yy)
    
    min_x = minimum(this_xx)
    min_y = minimum(this_yy)

    this_ww = -log10.(this_ww)
    
    this_ww -= minimum(this_ww)
    this_ww /= maximum(this_ww)
    
#     this_ss += 2 * this_ww - 1
    
#     this_ss += 2
    
    this_zz = -log10.(this_zz)
    
    Fussy.sort_lists!(this_zz,this_xx,this_yy,this_ss,this_mm,this_oo,this_ww)

    this_oo .*= (ceil.((0.6+0.4*this_ww)*10)/10)
#     this_ss = map(round,this_ss)

    cur_lims = [min_x, max_x, min_y, max_y]
    
    fix_lims!(cur_lims, x, y, xscale, yscale, deck)

    cur_labels = []
    for (tmp_index, tmp_label) in enumerate(simple ? ["kink", "kink-beta", "beta", "beta-wall", "wall"] : cur_list)
        cur_label = nothing
        if simple
            cur_label = tmp_label
        else
            
            cur_label = tmp_index < 3 ? "min $(latexstring(tmp_label))" : "$(tmp_label)-beta"
        end
        push!(cur_labels,cur_label)
        marker_size = 6
        ( markers[tmp_index] == :square ) && ( marker_size /= sqrt(2) )
        ( markers[tmp_index] == :diamond ) && ( marker_size /= sqrt(2) )
#         scatter!([],[],color=tmp_index < 3 ? tmp_index : tmp_index < 4  ? tmp_index + 1 : tmp_index + 3,label=cur_label, markershape=markers[tmp_index], markersize=6)
    end
    
    scatter!(this_xx, this_yy,zcolor=this_zz, color=:viridis, group=this_gg, colorbar_title=latexstring("-log( $z )"), markershape=:auto, label=String[cur_labels[sort(unique(this_gg))]...],markersize=6)#,marker=this_mm)#,markersize=this_ss+8, opacity=this_oo)#, markersize=marker_size),marker=markers[tmp_index], label=kind
  
    if in(x, cur_sampling.parameters)
        mid_x_value = cur_sampling.defaults[findfirst(cur_param -> x == cur_param, cur_sampling.parameters)]
        @assert mid_x_value >= 0
        
        min_x_value = mid_x_value * ( 1 - cur_sampling.sensitivity )
        max_x_value = mid_x_value * ( 1 + cur_sampling.sensitivity )
        
        plot!([1,1]*min_x_value, cur_lims[3:4], color=6, label="", style=:dash)
        plot!([1,1]*mid_x_value, cur_lims[3:4], color=5, label="", style=:dash)
        plot!([1,1]*max_x_value, cur_lims[3:4], color=6, label="", style=:dash)
    end
    
    if in(y, [:f_D, :f_BS, :f_CD, :f_IN, :norm_beta_N, :norm_P_W, :norm_q_95])
        plot!(cur_lims[1:2], [1,1], color=6, label="", style=:dash)
    end
    
    if in(x, [:f_D, :f_BS, :f_CD, :f_IN, :norm_beta_N, :norm_P_W, :norm_q_95])
        plot!([1,1], cur_lims[3:4], color=6, label="", style=:dash)
    end
    
    if in(y, cur_sampling.parameters)
        mid_y_value = cur_sampling.defaults[findfirst(cur_param -> y == cur_param, cur_sampling.parameters)]
        @assert mid_y_value >= 0
        
        min_y_value = mid_y_value * ( 1 - cur_sampling.sensitivity )
        max_y_value = mid_y_value * ( 1 + cur_sampling.sensitivity )
        
        plot!(cur_lims[1:2], [1,1]*min_y_value, color=6, label="", style=:dash)
        plot!(cur_lims[1:2], [1,1]*mid_y_value, color=5, label="", style=:dash)
        plot!(cur_lims[1:2], [1,1]*max_y_value, color=6, label="", style=:dash)
    end
    
    
    return cur_lims
end

return

In [12]:
@manipulate for mode=reverse(["comparisons", "sensitivities", "samplings"]), snap=false, legend=[true, false], xscale=["lin", "log"], yscale=["lin", "log"], уscale=["log", "lin"], simple=reverse([true,false]), Χ=x_list, X=xx_list, Х=xxx_list, У=y_list, Υ=yy_list, Y=yyy_list, deck = reverse(cur_decks), dummy=[true,false]    
    x = Χ
    xx = X 
    xxx = Х
    
    y = У
    yy = Υ
    yyy = Y
    
    tmp_plot = plot(legend=legend, html_output_format=:svg)
    
    hide_buttons = [-1,-5,-6,-7,-8,-3,-4,-9,-10,-11]
    
    if mode == "comparisons"
        cur_lims = make_comparisons(deck, x, y, xscale, yscale)
        xstring = string(x)
        ystring = string(y)
        
        show_buttons = [-8,-5,-11]
    elseif mode == "sensitivities"
        cur_lims = make_sensitivities(deck, xxx, yyy, xscale, yscale)
        xstring = string(xxx)
        ystring = string(yyy)
        
        show_buttons = [-6,-3,-11]
    else
        @assert mode == "samplings"
        cur_lims = make_samplings(deck, xx, yy, xscale, уscale, simple)
        xstring = string(xx)
        ystring = string(yy)
        
        show_buttons = [-7,-4,-9,-10]
    end
    
    if use_latex
        ( xstring == "cost" ) && ( xstring = "C_W")
        ( ystring == "cost" ) && ( ystring = "C_W")
        
        cur_xlabel = xstring
        cur_ylabel = ystring
        
        greek_letters = [ 
            "alpha", "beta", "gamma", "delta", 
            "epsilon", "zeta", "eta", "theta", 
            "iota", "kappa", "lambda", "mu", 
            "nu", "xi", "omicron", "pi", "rho", 
            "sigma", "tau", "upsilon", "phi", 
            "chi", "psi", "omega" 
        ]
        
        for cur_letter in greek_letters
            if cur_xlabel == cur_letter
                cur_xlabel = "\\$(cur_xlabel)"
            end
            
            if cur_ylabel == cur_letter
                cur_ylabel = "\\$(cur_ylabel)"
            end
            
            if endswith(cur_xlabel, "_$(cur_letter)")
                cur_xlabel = replace(cur_xlabel, "_$(cur_letter)", "_\\$(cur_letter)")
            end
            
            if endswith(cur_ylabel, "_$(cur_letter)")
                cur_ylabel = replace(cur_ylabel, "_$(cur_letter)", "_\\$(cur_letter)")
            end
            
            if startswith(cur_xlabel, "$(cur_letter)_")
                cur_xlabel = replace(cur_xlabel, "$(cur_letter)_", "\\$(cur_letter)_")
            end
            
            if startswith(cur_ylabel, "$(cur_letter)_")
                cur_ylabel = replace(cur_ylabel, "$(cur_letter)_", "\\$(cur_letter)_")
            end
            
            cur_xlabel = replace(cur_xlabel, "_$(cur_letter)_", "_\\$(cur_letter)_")
            cur_ylabel = replace(cur_ylabel, "_$(cur_letter)_", "_\\$(cur_letter)_")
        end
     
        if startswith(cur_xlabel, "norm_")
            cur_xlabel = "( $(cur_xlabel[6:end]) )_{norm}"
        else
            cur_xlabel = join(map(tmp_string -> "{$(tmp_string)}", split(cur_xlabel, "_")), "_")
        end

        if startswith(cur_ylabel, "norm_")
            cur_ylabel = "( $(cur_ylabel[6:end]) )_{norm}"
        else
            cur_ylabel = join(map(tmp_string -> "{$(tmp_string)}", split(cur_ylabel, "_")), "_")
        end
        
        if startswith(cur_xlabel, "max_")
            cur_xlabel = "( $(cur_xlabel[6:end]) )_{max}"
        else
            cur_xlabel = join(map(tmp_string -> "{$(tmp_string)}", split(cur_xlabel, "_")), "_")
        end

        if startswith(cur_ylabel, "max_")
            cur_ylabel = "( $(cur_ylabel[6:end]) )_{max}"
        else
            cur_ylabel = join(map(tmp_string -> "{$(tmp_string)}", split(cur_ylabel, "_")), "_")
        end
        
        if endswith(cur_xlabel, "_{bar}")
            cur_xlabel = "\\overline $(cur_xlabel[1:end-6])"
        end

        if endswith(cur_ylabel, "_{bar}")
            cur_ylabel = "\\overline $(cur_ylabel[1:end-6])"
        end
        
        cur_xlabel = latexstring(cur_xlabel)
        cur_ylabel = latexstring(cur_ylabel)
    else
        cur_xlabel = xstring
        cur_ylabel = ystring
    end
    
    xlabel!(cur_xlabel)
    ylabel!(cur_ylabel)
    
    toggle_buttons(show_buttons, hide_buttons)    
    
    cur_title = string(deck)
    cur_title = join(map(capitalize, split(cur_title, "_")), " ")
    cur_title = replace(cur_title, "1", "I")
    cur_title = replace(cur_title, "2", "II")
    
    title!(cur_title)
    
    xlims!(cur_lims[1:2]...)
    ylims!(cur_lims[3:4]...)

    if snap
        savefig(tmp_plot, "images/$(mode)/$(deck)_$(ystring)_vs_$(xstring).tex")
    end
    
    plot!()
end

Interact.Options{:ToggleButtons,String}(4: "input-2" = samplings String , "mode", "samplings", "samplings", 1, Interact.OptionDict(DataStructures.OrderedDict("samplings"=>"samplings","sensitivities"=>"sensitivities","comparisons"=>"comparisons"), Dict("sensitivities"=>"sensitivities","comparisons"=>"comparisons","samplings"=>"samplings")), Any[], Any[], true, "horizontal", true)

Interact.Checkbox(6: "input-3" = false Bool , "snap", false)

Interact.Options{:ToggleButtons,Bool}(7: "input-4" = true Bool , "legend", true, "true", 1, Interact.OptionDict(DataStructures.OrderedDict("true"=>true,"false"=>false), Dict(false=>"false",true=>"true")), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,String}(9: "input-5" = lin String , "xscale", "lin", "lin", 1, Interact.OptionDict(DataStructures.OrderedDict("lin"=>"lin","log"=>"log"), Dict("lin"=>"lin","log"=>"log")), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,String}(11: "input-6" = lin String , "yscale", "lin", "lin", 1, Interact.OptionDict(DataStructures.OrderedDict("lin"=>"lin","log"=>"log"), Dict("lin"=>"lin","log"=>"log")), Any[], Any[], true, "horizontal", true)

...

Interact.Options{:ToggleButtons,Symbol}(27: "input-14" = h_CS Symbol , "Y", :h_CS, "h_CS", 1, Interact.OptionDict(DataStructures.OrderedDict("h_CS"=>:h_CS,"a"=>:a,"b"=>:b,"B_0"=>:B_0,"beta_N"=>:beta_N,"c"=>:c,"cost"=>:cost,"d"=>:d,"eta_CD"=>:eta_CD,"f_BS"=>:f_BS…), Dict(:f_CD=>"f_CD",:b=>"b",:c=>"c",:q_95=>"q_95",:beta_N=>"beta_N",:tau_E=>"tau_E",:R_0=>"R_0",:W_M=>"W_M",:norm_P_E=>"norm_P_E",:eta_CD=>"eta_CD"…)), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,Symbol}(29: "input-15" = demo_pulsed Symbol , "deck", :demo_pulsed, "demo_pulsed", 1, Interact.OptionDict(DataStructures.OrderedDict("demo_pulsed"=>:demo_pulsed,"demo_steady"=>:demo_steady,"act_2"=>:act_2,"act_1"=>:act_1,"arc"=>:arc,"charybdis"=>:charybdis,"proteus"=>:proteus), Dict(:act_1=>"act_1",:proteus=>"proteus",:demo_steady=>"demo_steady",:charybdis=>"charybdis",:act_2=>"act_2",:arc=>"arc",:demo_pulsed=>"demo_pulsed")), Any[], Any[], true, "horizontal", true)

Interact.Options{:ToggleButtons,Bool}(31: "input-16" = true Bool , "dummy", true, "true", 1, Interact.OptionDict(DataStructures.OrderedDict("true"=>true,"false"=>false), Dict(false=>"false",true=>"true")), Any[], Any[], true, "horizontal", true)

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupy

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.109375 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 L 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 Z M 4.171875 0 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 L 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 Z M 4.46875 -1.734375 "/>
 
 
<path style="stroke:none;" d="M 7.046875 -3.34375 C 7.046875 -5.25 5.703125 -6.8125 4 -6.8125 L 0.34375 -6.8125 L 0.34375 -6.5 L 0.59375 -6.5 C 1.359375 -6.5 1.375 -6.390625 1.375 -6.03125 L 1.375 -0.78125 C 1.375 -0.421875 1.359375 -0.3125 0.59375 -0.3125 L 0.34375 -0.3125 L 0.34375 0 L 4 0 C 5.671875 0 7.046875 -1.46875 7.046875 -3.

In [ ]:
cur_script = """
    <script class="js-load-remove-tag">
        \$(".js-load-message").remove();
        \$("#appmode-busy").css("opacity","0");
        \$('pre:contains("WebIO: setting up")').parent(".output_subarea").remove();
        \$(".js-load-remove-tag").parent(".output_subarea").remove();
    </script>
"""

display(HTML(cur_script))

In [ ]:
cur_TT = linspace(0, 40)
cur_II = []
for cur_T in cur_TT
     tmp_reactor = Fussy.charybdis_deck(cur_T)
end

In [62]:
function makePlot(deck,mode,xxxx,yyyy,xscale="lin",yscale="lin")
    x = xxxx
    xx = xxxx
    xxx = xxxx
    
    y = yyyy
    yy = yyyy
    yyy = yyyy
    
    tmp_plot = plot(legend=true, html_output_format=:svg)
    
    hide_buttons = [-1,-5,-6,-7,-8,-3,-4,-9,-10,-11]
    
    if mode == "comparisons"
        cur_lims = make_comparisons(deck, x, y, xscale, yscale)
        xstring = string(x)
        ystring = string(y)
        
        show_buttons = [-8,-5,-11]
    elseif mode == "sensitivities"
        cur_lims = make_sensitivities(deck, xxx, yyy, xscale, yscale)
        xstring = string(xxx)
        ystring = string(yyy)
        
        show_buttons = [-6,-3,-11]
    else
        @assert mode == "samplings"
        cur_lims = make_samplings(deck, xx, yy, xscale, уscale, simple)
        xstring = string(xx)
        ystring = string(yy)
        
        show_buttons = [-7,-4,-9,-10]
    end
    
    if use_latex
        ( xstring == "cost" ) && ( xstring = "C_W")
        ( ystring == "cost" ) && ( ystring = "C_W")
        
        cur_xlabel = xstring
        cur_ylabel = ystring

        greek_letters = [ 
            "alpha", "beta", "gamma", "delta", 
            "epsilon", "zeta", "eta", "theta", 
            "iota", "kappa", "lambda", "mu", 
            "nu", "xi", "omicron", "pi", "rho", 
            "sigma", "tau", "upsilon", "phi", 
            "chi", "psi", "omega" 
        ]
        
        for cur_letter in greek_letters
            if cur_xlabel == cur_letter
                cur_xlabel = "\\$(cur_xlabel)"
            end
            
            if cur_ylabel == cur_letter
                cur_ylabel = "\\$(cur_ylabel)"
            end
            
            if endswith(cur_xlabel, "_$(cur_letter)")
                cur_xlabel = replace(cur_xlabel, "_$(cur_letter)", "_\\$(cur_letter)")
            end
            
            if endswith(cur_ylabel, "_$(cur_letter)")
                cur_ylabel = replace(cur_ylabel, "_$(cur_letter)", "_\\$(cur_letter)")
            end
            
            if startswith(cur_xlabel, "$(cur_letter)_")
                cur_xlabel = replace(cur_xlabel, "$(cur_letter)_", "\\$(cur_letter)_")
            end
            
            if startswith(cur_ylabel, "$(cur_letter)_")
                cur_ylabel = replace(cur_ylabel, "$(cur_letter)_", "\\$(cur_letter)_")
            end
            
            cur_xlabel = replace(cur_xlabel, "_$(cur_letter)_", "_\\$(cur_letter)_")
            cur_ylabel = replace(cur_ylabel, "_$(cur_letter)_", "_\\$(cur_letter)_")
        end
                
        if startswith(cur_xlabel, "norm_")
            cur_xlabel = "( $(cur_xlabel[6:end]) )_{norm}"
        elseif startswith(cur_xlabel, "max_")
            cur_xlabel = "( $(cur_xlabel[5:end]) )_{max}"
        else
            cur_xlabel = join(map(tmp_string -> "{$(tmp_string)}", split(cur_xlabel, "_")), "_")
        end

        if startswith(cur_ylabel, "norm_")
            tmp_label = join(map(tmp_string -> "{$(tmp_string)}", split(cur_ylabel[6:end], "_")), "_")
            
            cur_ylabel = "( $tmp_label )_{norm}"
        elseif startswith(cur_ylabel, "max_")
            tmp_label = join(map(tmp_string -> "{$(tmp_string)}", split(cur_ylabel[5:end], "_")), "_")
            
            cur_ylabel = "( $tmp_label )_{max}"
        else
            cur_ylabel = join(map(tmp_string -> "{$(tmp_string)}", split(cur_ylabel, "_")), "_")
        end
        
        if endswith(cur_xlabel, "_{bar}")
            cur_xlabel = "\\overline $(cur_xlabel[1:end-6])"
        end

        if endswith(cur_ylabel, "_{bar}")
            cur_ylabel = "\\overline $(cur_ylabel[1:end-6])"
        end
        
        cur_xlabel = latexstring(cur_xlabel)
        cur_ylabel = latexstring(cur_ylabel)
    else
        cur_xlabel = xstring
        cur_ylabel = ystring
    end
    
    xlabel!(cur_xlabel)
    ylabel!(cur_ylabel)
    
    toggle_buttons(show_buttons, hide_buttons)    
    
    cur_title = string(deck)
    cur_title = join(map(capitalize, split(cur_title, "_")), " ")
    cur_title = replace(cur_title, "1", "I")
    cur_title = replace(cur_title, "2", "II")
    
    title!(cur_title)
    
    xlims!(cur_lims[1:2]...)
    ylims!(cur_lims[3:4]...)

    savefig(tmp_plot, "images/$(mode)/$(deck)_$(ystring)_vs_$(xstring).tex")
    
    println(123)
    plot!()
end

makePlot (generic function with 3 methods)

In [63]:
comparison_items = [ 
    :T_bar, :n_bar, :I_P, 
#     :R_0, 
    :p_bar,
    :tau_E, :eta_CD, :f_BS, 
#     :W_M, :cost, 
    :q_DV, :norm_beta_N, :norm_q_95, :norm_P_W, 
#     :P_F,
    :b, :c, :d, :h_CS, :R_CS
]

for cur_comparison_item in comparison_items
    for cur_deck in cur_decks
        makePlot(cur_deck,"comparisons", :B_0, cur_comparison_item)
    end
end

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

123
123


HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

...

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

123
123


HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

123
123


In [64]:
sensitivity_items = [
#     :H, 
    :Q, :tau_FT,
    :N_G, :f_D, :Z_eff,
    :epsilon, :kappa_95, :delta_95,
    :nu_n, :nu_T, 
#     :l_i,
    :max_beta_N, :max_q_95, :max_P_W, 
]

for cur_sensitivity_item in sensitivity_items
    for cur_deck in cur_decks
        makePlot(cur_deck,"sensitivities", cur_sensitivity_item, :cost)
    end
end

plot!()

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

123
123


HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

...

123
123


HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

HTML{String}("    <script class=\"js-toggle\">\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-1)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-5)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-7)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-8)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-4)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-9)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-10)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().hide()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-6)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-3)).next().show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).show()\n        \$(\".jupyter-widgets\").closest(\".output_area\").eq(    \$(\".jupyter-widgets\").closest(\".output_area\").length\n+(-11)).next().show()\n        \$(\".js-toggle\").parent(\".output_subarea\").parent(\".output_area\").remove();\n    </script>\n")

123
123
123


<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 
 
 
 
<path style="stroke:none;" d="M 4.578125 -3.1875 C 4.578125 -3.984375 4.53125 -4.78125 4.1875 -5.515625 C 3.734375 -6.484375 2.90625 -6.640625 2.5 -6.640625 C 1.890625 -6.640625 1.171875 -6.375 0.75 -5.453125 C 0.4375 -4.765625 0.390625 -3.984375 0.390625 -3.1875 C 0.390625 -2.4375 0.421875 -1.546875 0.84375 -0.78125 C 1.265625 0.015625 2 0.21875 2.484375 0.21875 C 3.015625 0.21875 3.78125 0.015625 4.21875 -0.9375 C 4.53125 -1.625 4.578125 -2.40625 4.578125 -3.1875 Z M 3.765625 -3.3125 C 3.765625 -2.5625 3.765625 -1.890625 3.65625 -1.25 C 3.5 -0.296875 2.9375 0 2.484375 0 C 2.09375 0 1.5 -0.25 1.328125 -1.203125 C 1.21875 -1.796875 1.21875 -2.71875 1.21875 -3.3125 C 1.21875 -3.953125 1.21875 -4.609375 1.296875 -5.140625 C 1.484375 -6.328125 2.234375 -6.421875 2.484375 -6.421875 C 2.8125 -6.421875 3.46875 -6.234375 3.65625 -5.25 C 3.765625 -4.6875 3.765625 -3.9375 3.765625 -3.3125 Z M 3.765625 -3.3125 "/>
 
 
<path style="stroke:none;" d="M 4.46875 -2 C 4.46875 -3.1875 3.65625 -4.1875 2.578125 -4.1875 C 2.109375 -4.1875 1.671875 -4.03125 1.3125 -3.671875 L 1.3125 -5.625 C 1.515625 -5.5625 1.84375 -5.5 2.15625 -5.5 C 3.390625 -5.5 4.09375 -6.40625 4.09375 -6.53125 C 4.09375 -6.59375 4.0625 -6.640625 3.984375 -6.640625 C 3.984375 -6.640625 3.953125 -6.640625 3.90625 -6.609375 C 3.703125 -6.515625 3.21875 -6.3125 2.546875 -6.3125 C 2.15625 -6.3125 1.6875 -6.390625 1.21875 -6.59375 C 1.140625 -6.625 1.109375 -6.625 1.109375 -6.625 C 1 -6.625 1 -6.546875 1 -6.390625 L 1 -3.4375 C 1 -3.265625 1 -3.1875 1.140625 -3.1875 C 1.21875 -3.1875 1.234375 -3.203125 1.28125 -3.265625 C 1.390625 -3.421875 1.75 -3.96875 2.5625 -3.96875 C 3.078125 -3.96875 3.328125 -3.515625 3.40625 -3.328125 C 3.5625 -2.953125 3.59375 -2.578125 3.59375 -2.078125 C 3.59375 -1.71875 3.59375 -1.125 3.34375 -0.703125 C 3.109375 -0.3125 2.734375 -0.0625 2.28125 -0.0625 C 1.5625 -0.0625 0.984375 -0.59375 0.8125 -1.171875 C 0.84375 -1.171875 0.875 -1.15625 0.984375 -1.15625 C 1.3125 -1.15625 1.484375 -1.40625 1.484375 -1.640625 C 1.484375 -1.890625 1.3125 -2.140625 0.984375 -2.140625 C 0.84375 -2.140625 0.5 -2.0625 0.5 -1.609375 C 0.5 -0.75 1.1875 0.21875 2.296875 0.21875 C 3.453125 0.21875 4.46875 -0.734375 4.46875 -2 Z M 4.46875 -2 "/>
 
 
<path style="stroke:none;" d="M 4.171875 0 L 4.171875 -0.3125 L 3.859375 -0.3125 C 2.953125 -0.3125 2.9375 -0.421875 2.9375 -0.78125 L 2.9375 -6.375 C 2.9375 -6.625 2.9375 -6.640625 2.703125 -6.640625 C 2.078125 -6 1.203125 -6 0.890625 -6 L 0.890625 -5.6875 C 1.09375 -5.6875 1.671875 -5.6875 2.1875 -5.953125 L 2.1875 -0.78125 C 2.1875 -0.421875 2.15625 -0.3125 1.265625 -0.3125 L 0.953125 -0.3125 L 0.953125 0 C 1.296875 -0.03125 2.15625 -0.03125 2.5625 -0.03125 C 2.953125 -0.03125 3.828125 -0.03125 4.171875 0 Z M 4.171875 0 "/>
 
 
 
 
 
<path style="stroke:none;" d="M 4.46875 -1.734375 L 4.234375 -1.734375 C 4.171875 -1.4375 4.109375 -1 4 -0.84375 C 3.9375 -0.765625 3.28125 -0.765625 3.0625 -0.765625 L 1.265625 -0.765625 L 2.328125 -1.796875 C 3.875 -3.171875 4.46875 -3.703125 4.46875 -4.703125 C 4.46875 -5.84375 3.578125 -6.640625 2.359375 -6.640625 C 1.234375 -6.640625 0.5 -5.71875 0.5 -4.828125 C 0.5 -4.28125 1 -4.28125 1.03125 -4.28125 C 1.203125 -4.28125 1.546875 -4.390625 1.546875 -4.8125 C 1.546875 -5.0625 1.359375 -5.328125 1.015625 -5.328125 C 0.9375 -5.328125 0.921875 -5.328125 0.890625 -5.3125 C 1.109375 -5.96875 1.65625 -6.328125 2.234375 -6.328125 C 3.140625 -6.328125 3.5625 -5.515625 3.5625 -4.703125 C 3.5625 -3.90625 3.078125 -3.125 2.515625 -2.5 L 0.609375 -0.375 C 0.5 -0.265625 0.5 -0.234375 0.5 0 L 4.203125 0 Z M 4.46875 -1.734375 "/>
 
 
<path style="stroke:none;" d="M 4.6875 -1.640625 L 4.6875 -1.953125 L 3.703125 -1.953125 L 3.703125 -6.484375 C 3.703125 -6.6875 3.703125 -6.75 3.53125 -6.75 C 3.453125 -6.75 3.421875 -6.75 3.34375 -6.625 L 0.28125 -1.953125 L 0.28125 -1.640625 L 2.9375 -1.640625 L 2.9375 -0.78125 C 2.9375 -0.421875 2.90625 -0.3125 2.171875 -0.3

In [22]:
@manipulate for cur_deck in cur_decks
    cur_reactors = filter(tmp_reactor -> tmp_reactor.is_valid, cur_scans[cur_deck].beta_reactors)
    cur
end


Interact.Options{:ToggleButtons,Symbol}(47: "input-21" = proteus Symbol , "cur_deck", :proteus, "proteus", 1, Interact.OptionDict(DataStructures.OrderedDict("proteus"=>:proteus,"charybdis"=>:charybdis,"arc"=>:arc,"act_1"=>:act_1,"act_2"=>:act_2,"demo_steady"=>:demo_steady,"demo_pulsed"=>:demo_pulsed), Dict(:act_1=>"act_1",:proteus=>"proteus",:demo_steady=>"demo_steady",:charybdis=>"charybdis",:act_2=>"act_2",:arc=>"arc",:demo_pulsed=>"demo_pulsed")), Any[], Any[], true, "horizontal", true)

189-element Array{Fussy.Reactor,1}:
 Fussy.Reactor
  T_bar: Float64 0.333333
  n_bar: Float64 445.489
  I_P: Float64 0.123176
  R_0: Float64 0.0277295
  B_0: Float64 32.2282
  sigma_v: Float64 6.79926e-8
  mode_scaling: Dict{Symbol,Float64}
  deck: Symbol proteus
  is_pulsed: Bool true
  is_symbolic: Bool false
  is_consistent: Bool false
  is_solved: Bool true
  is_valid: Bool false
  is_good: Bool true
  H: Float64 1.0
  Q: Float64 25.0
  epsilon: Float64 0.3
  kappa_95: Float64 1.8
  delta_95: Float64 0.35
  nu_n: Float64 0.4
  nu_T: Float64 1.1
  Z_eff: Float64 1.75
  f_D: Float64 0.9
  A: Float64 2.5
  l_i: Float64 0.632845
  rho_m: Float64 0.000686639
  gamma: Float64 1.0
  N_G: Float64 0.9
  eta_T: Float64 0.4
  eta_RF: Float64 0.5
  tau_FT: Float64 7200.0
  C_ejima: Float64 0.3
  pi: Float64 3.14159
  speed_of_light: Float64 3.0e8
  B_CS: Float64 20.0
  k: Float64 0.4
  sigma_CS: Float64 600.0
  sigma_TF: Float64 300.0
  J_CS: Float64 50.0
  J_TF: Float64 200.0
  max_beta_N: Fl

In [18]:
cur_scans

Dict{Any,Any} with 23 entries:
  :charybdis_simple     => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :charybdis            => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :act_1                => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :demo_pulsed_simple   => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :demo_steady_modified => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :arc_simple           => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :proteus_simple       => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :demo_pulsed          => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :act_1_simple         => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :act_2                => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :act_2_simple         => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1.…
  :demo_pulsed_modified => Fussy.Scan(AbstractFloat[0.0, 0.333333, 0.666667, 1